In [1]:
import numpy as np
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Training Set Preparation**

In [4]:
df_train_raw = pd.read_csv("/content/drive/MyDrive/HocMayThongKe/Project/Raw Dataset/Train.csv")
df_train_raw.head()

,index,comment,n_star,date_time,label
0,0,Mới mua máy này Tại thegioididong thốt nốt cảm...,5,2 tuần trước,{CAMERA#Positive};{FEATURES#Positive};{BATTERY...
1,1,Pin kém còn lại miễn chê mua 8/3/2019 tình trạ...,5,14/09/2019,{BATTERY#Negative};{GENERAL#Positive};{OTHERS};
2,2,Sao lúc gọi điện thoại màn hình bị chấm nhỏ nh...,3,17/08/2020,{FEATURES#Negative};
3,3,"Mọi người cập nhật phần mềm lại , nó sẽ bớt tố...",3,29/02/2020,{FEATURES#Negative};{BATTERY#Neutral};{GENERAL...
4,4,"Mới mua Sài được 1 tháng thấy pin rất trâu, Sà...",5,4/6/2020,{BATTERY#Positive};{PERFORMANCE#Positive};{SER...


In [5]:
df_train_raw = df_train_raw.drop(columns=['n_star','index', 'date_time'])
df_train_raw.head()

,comment,label
0,Mới mua máy này Tại thegioididong thốt nốt cảm...,{CAMERA#Positive};{FEATURES#Positive};{BATTERY...
1,Pin kém còn lại miễn chê mua 8/3/2019 tình trạ...,{BATTERY#Negative};{GENERAL#Positive};{OTHERS};
2,Sao lúc gọi điện thoại màn hình bị chấm nhỏ nh...,{FEATURES#Negative};
3,"Mọi người cập nhật phần mềm lại , nó sẽ bớt tố...",{FEATURES#Negative};{BATTERY#Neutral};{GENERAL...
4,"Mới mua Sài được 1 tháng thấy pin rất trâu, Sà...",{BATTERY#Positive};{PERFORMANCE#Positive};{SER...


In [6]:
aspect_cols = ['GENERAL', 'SCREEN', 'CAMERA', 'FEATURES', 'BATTERY', 'PERFORMANCE', 'STORAGE', 'DESIGN', 'PRICE', 'SER&ACC']

In [7]:
raw_labels = df_train_raw['label'].values

aspect_values = []
other_values = []

for row in raw_labels:
    brackets = row.split(';')
    if brackets[-2] == '{OTHERS}':
      other_values.append(1)
    else:
      other_values.append(0)

    tmp = ['Neutral'] * 10
    for cur_bracket in brackets:
        if '#' in cur_bracket:
            cur_aspect, value = cur_bracket.strip('{}').split('#')
            tmp[aspect_cols.index(cur_aspect)] = value

    aspect_values.append(tmp)

In [8]:
df_aspects = pd.DataFrame(aspect_values, columns=aspect_cols)
df_others = pd.DataFrame(other_values, columns=['OTHERS'])

In [9]:
df_train = pd.concat([df_train_raw.drop(columns=['label']), df_aspects, df_others], axis=1)
df_train.head()

,comment,GENERAL,SCREEN,CAMERA,FEATURES,BATTERY,PERFORMANCE,STORAGE,DESIGN,PRICE,SER&ACC,OTHERS
0,Mới mua máy này Tại thegioididong thốt nốt cảm...,Positive,Neutral,Positive,Positive,Positive,Neutral,Neutral,Neutral,Positive,Positive,0
1,Pin kém còn lại miễn chê mua 8/3/2019 tình trạ...,Positive,Neutral,Neutral,Neutral,Negative,Neutral,Neutral,Neutral,Neutral,Neutral,1
2,Sao lúc gọi điện thoại màn hình bị chấm nhỏ nh...,Neutral,Neutral,Neutral,Negative,Neutral,Neutral,Neutral,Neutral,Neutral,Neutral,0
3,"Mọi người cập nhật phần mềm lại , nó sẽ bớt tố...",Neutral,Neutral,Neutral,Negative,Neutral,Neutral,Neutral,Neutral,Neutral,Neutral,0
4,"Mới mua Sài được 1 tháng thấy pin rất trâu, Sà...",Neutral,Neutral,Neutral,Neutral,Positive,Positive,Neutral,Neutral,Neutral,Negative,0


In [10]:
sentiment_mapping = {'Positive': 1, 'Negative': -1, 'Neutral': 0}
for col in aspect_cols:
    df_train[col] = df_train[col].map(sentiment_mapping)

df_train.head()

,comment,GENERAL,SCREEN,CAMERA,FEATURES,BATTERY,PERFORMANCE,STORAGE,DESIGN,PRICE,SER&ACC,OTHERS
0,Mới mua máy này Tại thegioididong thốt nốt cảm...,1,0,1,1,1,0,0,0,1,1,0
1,Pin kém còn lại miễn chê mua 8/3/2019 tình trạ...,1,0,0,0,-1,0,0,0,0,0,1
2,Sao lúc gọi điện thoại màn hình bị chấm nhỏ nh...,0,0,0,-1,0,0,0,0,0,0,0
3,"Mọi người cập nhật phần mềm lại , nó sẽ bớt tố...",0,0,0,-1,0,0,0,0,0,0,0
4,"Mới mua Sài được 1 tháng thấy pin rất trâu, Sà...",0,0,0,0,1,1,0,0,0,-1,0


# **Testing Set Preparation**

In [11]:
df_test_raw = pd.read_csv("/content/drive/MyDrive/HocMayThongKe/Project/Raw Dataset/Test.csv")
df_test_raw = df_test_raw.drop(columns=['n_star','index', 'date_time'])
df_test_raw.head()

,comment,label
0,"Điện thoải ổn. Facelock cực nhanh, vân tay ôk ...",{SCREEN#Positive};{FEATURES#Positive};{PERFORM...
1,"Mình mới mua vivo91c. Tải ứng dụng ,games nh...",{FEATURES#Negative};{PERFORMANCE#Positive};{SE...
2,Xấu đẹp gì ko biết nhưng rất ưng TGdđ phục vụ ...,{DESIGN#Neutral};{SER&ACC#Positive};
3,Màn hình hơi lác khi chơi game. Game nặng thì ...,{PERFORMANCE#Negative};{DESIGN#Negative};{OTHE...
4,"Nói chung máy đẹp với màn amoled, ổn trong tầm...",{SCREEN#Positive};{BATTERY#Negative};{DESIGN#P...


In [12]:
raw_labels = df_test_raw['label'].values

aspect_values = []
other_values = []

for row in raw_labels:
    brackets = row.split(';')
    if brackets[-2] == '{OTHERS}':
      other_values.append(1)
    else:
      other_values.append(0)

    tmp = ['Neutral'] * 10
    for cur_bracket in brackets :
        if '#' in cur_bracket:
            cur_aspect, value = cur_bracket.strip('{}').split('#')
            tmp[aspect_cols.index(cur_aspect)] = value

    aspect_values.append(tmp)

In [13]:
df_aspects = pd.DataFrame(aspect_values, columns=aspect_cols)
df_others = pd.DataFrame(other_values, columns=['OTHERS'])

df_test = pd.concat([df_test_raw.drop(columns=['label']), df_aspects, df_others], axis=1)
df_test.head()

,comment,GENERAL,SCREEN,CAMERA,FEATURES,BATTERY,PERFORMANCE,STORAGE,DESIGN,PRICE,SER&ACC,OTHERS
0,"Điện thoải ổn. Facelock cực nhanh, vân tay ôk ...",Positive,Positive,Neutral,Positive,Neutral,Positive,Neutral,Neutral,Neutral,Neutral,0
1,"Mình mới mua vivo91c. Tải ứng dụng ,games nh...",Neutral,Neutral,Neutral,Negative,Neutral,Positive,Neutral,Neutral,Neutral,Positive,1
2,Xấu đẹp gì ko biết nhưng rất ưng TGdđ phục vụ ...,Neutral,Neutral,Neutral,Neutral,Neutral,Neutral,Neutral,Neutral,Neutral,Positive,0
3,Màn hình hơi lác khi chơi game. Game nặng thì ...,Neutral,Neutral,Neutral,Neutral,Neutral,Negative,Neutral,Negative,Neutral,Neutral,1
4,"Nói chung máy đẹp với màn amoled, ổn trong tầm...",Neutral,Positive,Neutral,Neutral,Negative,Neutral,Neutral,Positive,Neutral,Neutral,0


In [14]:
sentiment_mapping = {'Positive': 1, 'Negative': -1, 'Neutral': 0}
for col in aspect_cols:
    df_test[col] = df_test[col].map(sentiment_mapping)

df_test.head()

,comment,GENERAL,SCREEN,CAMERA,FEATURES,BATTERY,PERFORMANCE,STORAGE,DESIGN,PRICE,SER&ACC,OTHERS
0,"Điện thoải ổn. Facelock cực nhanh, vân tay ôk ...",1,1,0,1,0,1,0,0,0,0,0
1,"Mình mới mua vivo91c. Tải ứng dụng ,games nh...",0,0,0,-1,0,1,0,0,0,1,1
2,Xấu đẹp gì ko biết nhưng rất ưng TGdđ phục vụ ...,0,0,0,0,0,0,0,0,0,1,0
3,Màn hình hơi lác khi chơi game. Game nặng thì ...,0,0,0,0,0,-1,0,-1,0,0,1
4,"Nói chung máy đẹp với màn amoled, ổn trong tầm...",0,1,0,0,-1,0,0,1,0,0,0


# **Save preprocessed train and test set as csv file into Google Colab**

In [15]:
!ls '/content/drive/MyDrive/HocMayThongKe/Project/Preprocessed Dataset'

In [16]:
df_train.to_csv('/content/drive/MyDrive/HocMayThongKe/Project/Preprocessed Dataset/df_train.csv', index=False, mode='w')
df_test.to_csv('/content/drive/MyDrive/HocMayThongKe/Project/Preprocessed Dataset/df_test.csv', index=False, mode='w')

In [17]:
!ls '/content/drive/MyDrive/HocMayThongKe/Project/Preprocessed Dataset'

df_test.csv  df_train.csv
